In [245]:
import random

class Punto:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, B):
        if  isinstance(B, Punto):
            return (self.x == B.x) and (self.y == B.y)
        return False
        
    def __hash__(self):
        return hash((self.x, self.y))

    def arriba(self):
        return Punto(self.x, self.y + 1)
        
    def abajo(self):
        return Punto(self.x, self.y - 1)
        
    def derecha(self):
        return Punto(self.x + 1, self.y)
        
    def izquierda(self):
        return Punto(self.x - 1, self.y)
        
    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
        
    def __repr__(self):
        return str(self)

    def distancia(self, A):
        return abs(self.x-A.x) + abs(self.y-A.y)
        
    def __sub__(self, A):
        assert A, "No es posible restar"
        return Punto(self.x - A.x, self.y - A.y)
    def distancia(self, B):
        assert isinstance(B, Punto), "B debe ser punto"
        return abs(self.x - B.x) + abs(self.y - B.y)

In [260]:
class Camino:
    def __init__(self, entrada):
        espunto = isinstance(entrada, Punto)
        eslista = isinstance(entrada, list)
        eslistcamino = True
        if eslista:
            #revizamos si todos son puntos
            eslistapuntos = all([isinstance(punto, Punto) for punto in entrada])
            #ahora si estan a distancia uno
            for i in range(len(entrada)-1):
                eslistapuntos = eslistapuntos and (Punto.distancia(entrada[i], entrada[i+1]) == 1)
                
        assert  espunto or (eslista and eslistcamino), "debe ser lista continua o punto"
        #assert all([isinstance(punto, Punto) for punto in entrada]), "la lista debe ser de puntos"
        #revizar que puntos esten a distancia 1
        
        if espunto:
            self.camino = [entrada]
        else:
            self.camino = list(entrada)
            
    def __str__(self):
        return "".join(self.strCamino())
        
    def __repr__(self):
        return str(self)
        
    def agregarPunto(self, A):
        assert isinstance(A, Punto), "Tas mal"
        assert self.camino[-1].distancia(A) == 1, "El punto a agregar debe estar pegado al final del camino"
        self.camino.append(A)

    def getPuntos(self):
        return list(self.camino)
        
    def __contains__(self, A):
        #agregar condiciones
        return A in self.getPuntos()

    def strCamino(self):
        """
        regresa una lista con strings de felchas indicando en que direccion fue cada movimiento del camina
        """
        if len(self.camino) == 1:
            return ["•"]
        camino = []

        #la primera flecha es distinta
        if ((self.camino[1] - self.camino[0]) == Punto(1, 0)):
            camino.append("↦")
        elif ((self.camino[1] - self.camino[0]) == Punto(0, 1)):
            camino.append("↧")
        elif ((self.camino[1] - self.camino[0]) == Punto(-1, 0)):
            camino.append("↤")
        else:
            camino.append("↥")
            
        for i in range(1,len(self.camino)-1):
            if ((self.camino[i + 1] - self.camino[i]) == Punto(1, 0)):
                camino.append("→")
            elif ((self.camino[i + 1] - self.camino[i]) == Punto(0, 1)):
                camino.append("↓")
            elif ((self.camino[i + 1] - self.camino[i]) == Punto(-1, 0)):
                camino.append("←")
            else:
                camino.append("↑")
                
        camino.append("•")
        return camino
    def __len__(self):
        return len(self.camino)
    def __getitem__(self, key):
        esslice = isinstance(key, slice)
        esint = isinstance(key, int)
        assert esslice or esint, "error"   
        return Camino(self.camino[key])

        

In [418]:
class Plano:
    def __init__(self, x, y, inicio = None, final = None, huecos = set()):
        if not (inicio is None):
            assert inicio in plano, "inicio no esta en el plano"
        if not(final is None):
            assert final in plano, "final no esta en el plano"
        #Agregar asserts      
        self.lenx = x
        self.leny = y
        self.puntos = [[Punto(x1,y1) for x1 in range(int(x)+1)]  for y1 in range(int(y+1))]
        
        self.huecos = [hueco for hueco in huecos if hueco in self]

        for hueco in self.huecos:
            self.puntos[hueco.y][hueco.x] = None
        
        if not (inicio is None):
            self.inicio = inicio 
        else:
           self.inicio = random.choice(self.getPuntos()) 
        #cambiar nombre
        
        lista = self.getPuntos()
        lista.remove(self.inicio)
        
        if not (final is None):
            self.final = final 
        else:
           self.final = random.choice(lista) 
        
    def distancia(self, A, B):
        #considerar huecos
        assert isinstance(A, Punto) and isinstance(B, punto), "entradas deben ser de tipo punto"    
        return abs(A.x-B.x) + abs(A.y-B.y)

    def puntosCercanos(self, A): 
        assert A in self, "Este punto no esta en el plano"
        puntosCercanos = [] #cambiar a set?
        puntosAdjacentos = {A.arriba(), A.abajo(), A.derecha(),A.izquierda}
        
        for punto in puntosAdjacentos:
            if punto in self:
                puntosCercanos.append(punto)
                
        return puntosCercanos
                
    

    def getPuntos(self):
        #mejorar
        puntos = []
        for lista_puntos in self.puntos:
            for punto in lista_puntos:
                puntos.append(punto)
        return puntos

    def __contains__(self, A):
        if isinstance(A, Punto):          
            return A in self.getPuntos()
        elif isinstance(A, Camino):
            return all([(B in self) for B in A.getPuntos()])
        return False

    def __str__(self):
        string = "   " + " ".join([str(n).ljust(2, " ") for n in range(self.lenx + 1)]) + "\n"
        for y in range(self.leny + 1): 
            string = (string + str(y)) if len(str(y)) == 2 else (string + " "+ str(y))
            for x in range(self.lenx + 1):
                
                if not (self.puntos[y][x] is None):
                    string += " * "
                else:
                    string += " x "
            string = string + "\n" 
        return string
        
    def __repr__(self):
        return str(self)

    def __getitem__(self, indices):
        
        # Personaliza cómo se acceden los elementos de la instancia.
        return Plano(indices[0], indices[1], huecos = self.huecos)

    def hacerHueco(self, A):
        assert isinstance(A, Punto), "No es un punto"
        self.puntos[A.y][A.x] = None

    def solucionAleatoria(self, camino = None):
        #creamos objetoa camino que empieza en el inicio 
        if camino is None:
            camino = Camino(self.inicio)
        #lista de puntos a distancia 1
        puntosQuePuedenSerOpciones = [camino.getPuntos()[-1].arriba(),
                                      camino.getPuntos()[-1].abajo(), 
                                      camino.getPuntos()[-1].izquierda(), 
                                      camino.getPuntos()[-1].derecha()]
        
        #revizamos cuales estan en el plano
        
        puntosPosibles = [punto for punto in puntosQuePuedenSerOpciones if ((punto in self) and not(punto in camino))]
        
        if puntosPosibles:
            
            #seleccionamos uno
            puntoNuevo = random.choice(puntosPosibles)
            
            camino.agregarPunto(puntoNuevo)
            return camino if self.final == camino.getPuntos()[-1] else self.solucionAleatoria(camino = camino)
            
        return camino
        
    def printCamino(self, camino):
        string = "   " + " ".join([str(n).ljust(2, " ") for n in range(self.lenx + 1)]) + "\n"
        simbolos = camino.strCamino()
        
        for y in range(self.leny + 1 ):
            string = (string + str(y)) if len(str(y)) == 2 else (string + " "+ str(y)) 
            for x in range(self.lenx + 1):
                if not (self.puntos[y][x] is None):
                    if self.puntos[y][x] in camino:
                        string += " " + simbolos[(camino.getPuntos()).index(self.puntos[y][x])] + " "
                    else:
                        string += " * "
                else:
                    string += " x "
            string += "\n"
        print(string)
    def solucionesAleatorias(self, size):
        assert isinstance(size,int), "size debe ser entero"
        return [self.solucionAleatoria() for a in range(size)]
        
    def costo(self, camino):
        escamino = isinstance(camino, Camino)
        eslista = isinstance(camino, list)
        eslistadepuntos = True
        
        if eslista:
            eslistadepuntos = all([isinstance(caminon, Camino) for caminon in camino])
        
        assert (escamino or (eslista and eslistadepuntos)), "Error"

        if escamino:
            assert camino in self, "Este camino no esta en el plano"
        
        if escamino:
            if camino[-1] != self.final:
                return float("inf")
            return len(camino)

        return [self.costo(caminon) for caminon in camino]

    def solucionI(self, camino = None):
        """
            regresa una solucion que intenta llegar al final con un algoritmo muy malo por el momento
        """
        if camino is None:
            camino = Camino(self.inicio)

        if camino.getPuntos()[-1] == self.final:
            return camino
        #puntos cercan
        pcaminofinal =camino.getPuntos()[-1]
        parriba = pcaminofinal.arriba()
        pabajo = pcaminofinal.abajo()
        pderecha = pcaminofinal.derecha()
        pizquierda = pcaminofinal.izquierda()
        derechaoizquierdaposible = True
        algunofueposible = True
        #    pcaminofinal - 1]
        derecha = self.final.x - pcaminofinal.x > 0#False if (pcaminofinal.x - self.final.x > 0) else True
        izquierda =  self.final.x - pcaminofinal.x < 0#False if (pcaminofinal.x - self.final.x < 0) else True 
        arriba = self.final.y - pcaminofinal.y > 0#False if (pcaminofinal.y - self.final.y > 0) else True
        abajo = self.final.y - pcaminofinal.y < 0#False if (pcaminofinal.y - self.final.y < 0) else True
        
        #primero vertical
       ?" abajo: " + str(pabajo) + " izquierda: " + str(pizquierda) + " derecha: " + str(pderecha))
        if derecha:
            if (not (pderecha in camino)) and (pderecha in self):
                algunofueposible = False
                derechaoizquierdaposible = False
                #print("Se va a agregar derecha" + str(pderecha))
                camino.agregarPunto(pderecha)
                
        if izquierda:
            if (not (pizquierda in camino)) and (pizquierda in self):
                algunofueposible = False
                derechaoizquierdaposible = False
                #print("Se va a agregar izquierda" + str(pizquierda))
                camino.agregarPunto(pizquierda)

        #si no agregamos vertical, vemos arriba o abajo
        
        if derechaoizquierdaposible:
            if arriba:
                if (not (parriba in camino)) and ( parriba in self):
                    algunofueposible = False
                    #print("Se va a agregar arriba" + str(parriba))
                    camino.agregarPunto(parriba)
            if abajo:
                if (not (pabajo in camino)) and (pabajo in self):
                    algunofueposible = False
                    #print("Se va a agregar abajo" + str(pabajo))
                    camino.agregarPunto(pabajo)
                
        #Si no nos pudimos mover a uno mejor, tomamos uno que si se pueda aleatorio
        puntosQuePuedenSerOpciones = [parriba,
                                      pabajo, 
                                      pizquierda, 
                                      pderecha]
        
        #revizamos cuales estan en el plano
        #si no puede caminar a la mejor opcion posible, entonces que se vaya a donde pueda
        puntosPosibles = [punto for punto in puntosQuePuedenSerOpciones if ((punto in self) and not(punto in camino))]
        
        if algunofueposible:
            
            #seleccionamos uno
            #print("Puntos posibles: " + str(puntosPosibles))
            puntoNuevo = random.choice(puntosPosibles)
            
            camino.agregarPunto(puntoNuevo)
            return self.solucionI(camino = camino)
            
        return self.solucionI(camino = camino)


                                            
                                                                             
                                                                             
            
        
            
            
    

In [423]:
plano = Plano(20,20, 
              huecos = [Punto(5, n) for n in range(20)],
              inicio = Punto(0,0), 
              final = Punto(20,20))


In [424]:
camino1 = plano.solucionAleatoria()

In [425]:
print("el inicio es: " + str(plano.inicio) + ", el final es: " + str(plano.final))

el inicio es: (0, 0), el final es: (20, 20)


In [426]:
plano

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20
 0 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 1 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 2 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 3 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 4 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 5 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 6 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 7 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 8 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 9 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
10 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
11 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
12 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
13 *  *  *  *  *  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
14 *  *  *

In [427]:
caminofuncional = plano.solucionI()

empieza iteracion
Estamos en: (0, 0)
arriba: True abajo: False izquierda: False derecha: True
Puntos posibles a distancia 1 son arriba: (0, 1) abajo: (0, -1) izquierda: (-1, 0) derecha: (1, 0)
Se va a agregar derecha(1, 0)
empieza iteracion
Estamos en: (1, 0)
arriba: True abajo: False izquierda: False derecha: True
Puntos posibles a distancia 1 son arriba: (1, 1) abajo: (1, -1) izquierda: (0, 0) derecha: (2, 0)
Se va a agregar derecha(2, 0)
empieza iteracion
Estamos en: (2, 0)
arriba: True abajo: False izquierda: False derecha: True
Puntos posibles a distancia 1 son arriba: (2, 1) abajo: (2, -1) izquierda: (1, 0) derecha: (3, 0)
Se va a agregar derecha(3, 0)
empieza iteracion
Estamos en: (3, 0)
arriba: True abajo: False izquierda: False derecha: True
Puntos posibles a distancia 1 son arriba: (3, 1) abajo: (3, -1) izquierda: (2, 0) derecha: (4, 0)
Se va a agregar derecha(4, 0)
empieza iteracion
Estamos en: (4, 0)
arriba: True abajo: False izquierda: False derecha: True
Puntos posibles a

In [ ]:
caminofuncional.getPuntos()

In [428]:
plano.printCamino(caminofuncional)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20
 0 ↦  →  →  →  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 1 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 2 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 3 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 4 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 5 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 6 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 7 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 8 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 9 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
10 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
11 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
12 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
13 *  *  *  *  ↓  x  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
14 *  *  *